In [4]:
# Import WRDS library
import wrds
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os

In [67]:

def read_sql_script(fname):
    fd = open(fname, 'r')
    sqlFile = fd.read()
    fd.close()

    return sqlFile


# Get current path
this_path = os.path.abspath(os.curdir)
print(this_path)

# Define sql file names
# these will be used as a global variable
query_historical_prices = read_sql_script('/Users/abigail/Desktop/SMU/QF603/get_historical_prices.sql')
query_shares_outstanding = read_sql_script('/Users/abigail/Desktop/SMU/QF603/get_shares_outstanding.sql')
query_historical_one_price = read_sql_script('/Users/abigail/Desktop/SMU/QF603/get_historical_price_1d.sql')


# Establish live connection; requires user login (passwords will be masked)
db = wrds.Connection() # this will be used as a global variable


def get_historical_prices(isin, start_date, end_date):
    
    print(f'Extracting historical prices for {isin}...')

    df =\
    (
        db
        .raw_sql(
            query_historical_prices.format(isin, start_date, end_date), 
            date_cols = ['trade_date']
            )
    )

    if df.empty:
        print('Dataframe is empty. No results was returned!')
    
    print('--------------------------------------------------')

    return df



def get_historical_one_price(isin, date):
    
    print(f'Extracting historical prices for {isin}...')

    df =\
    (
        db
        .raw_sql(
            query_historical_one_price.format(isin, date), 
            date_cols = ['trade_date']
            )
    )

    if df.empty:
        print('Dataframe is empty. No results was returned!')
    
    print('--------------------------------------------------')

    return df

/Users/abigail/Desktop/SMU/QF603
Enter your WRDS username [abigail]:abigailcjh
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [17]:
#first iteration

ftse_rebal = pd.read_csv('ftse100_rebalancing_summary.csv')


ftse_rebal["post_date"] =\
    (
        ftse_rebal["Post Date"]
        .apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
    )

#pd.to_datetime(series, format=)


ftse_rebal["start_date"] =\
    (
        ftse_rebal["post_date"] - timedelta(days = 7)
    )
    
ftse_rebal["end_date"] =\
    (
        ftse_rebal["post_date"] + timedelta(days = 28)
    )

ftse_rebal["fmt_start_date"] =\
    (
        ftse_rebal["start_date"]
        .apply(lambda x: datetime.strftime(x,"%d/%m/%Y"))
    )
    
ftse_rebal["fmt_end_date"] =\
    (
        ftse_rebal["end_date"]
        .apply(lambda x: datetime.strftime(x,"%d/%m/%Y"))
    )


target_isins = ftse_rebal["ISIN\n"]
start_dates = ftse_rebal["fmt_start_date"]
end_dates = ftse_rebal["fmt_end_date"]   
    
    


historical_prices =\
    (
        pd.
        concat(
            map(
                get_historical_prices,
                target_isins,
                start_dates,
                end_dates
            )
        )
    )

(
        historical_prices
        .to_csv(
            f'/Users/abigail/Desktop/SMU/QF603/output/historical_prices_ftse_rebal_20230923.csv',
            index = False
            )
)

print('===================================================')
    

In [68]:
#second iteration

ftse_full_rebal = pd.read_csv('ftse_largemid_rebalancing_summary.csv')


ftse_full_rebal["post_date"] =\
    (
        ftse_full_rebal["Post Date"]
        .apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
    )

#pd.to_datetime(series, format=)



ftse_full_rebal["start_date"] =\
    (
        ftse_full_rebal["post_date"] - timedelta(days = 7)
    )
    
ftse_full_rebal["end_date"] =\
    (
        ftse_full_rebal["post_date"] + timedelta(days = 28)
    )

ftse_full_rebal["fmt_start_date"] =\
    (
        ftse_full_rebal["start_date"]
        .apply(lambda x: datetime.strftime(x,"%d/%m/%Y"))
    )
    
ftse_full_rebal["fmt_end_date"] =\
    (
        ftse_full_rebal["end_date"]
        .apply(lambda x: datetime.strftime(x,"%d/%m/%Y"))
    )




target_isins = ftse_full_rebal["ISIN\n"]
start_dates = ftse_full_rebal["fmt_start_date"]
end_dates = ftse_full_rebal["fmt_end_date"]   



historical_prices =\
    (
        pd.
        concat(
            map(
                get_historical_prices,
                target_isins,
                start_dates,
                end_dates
            )
        )
    )

(
        historical_prices
        .to_csv(
            f'/Users/abigail/Desktop/SMU/QF603/output/historical_prices_ftse_full_rebal_20230923.csv',
            index = False
            )
)

print('===================================================')
    

Extracting historical prices for GB0030927254...
--------------------------------------------------
Extracting historical prices for GB00BF4HYT85...
--------------------------------------------------
Extracting historical prices for GB00BMMV6B79...
Dataframe is empty. No results was returned!
--------------------------------------------------
Extracting historical prices for GB00B033F229...
--------------------------------------------------
Extracting historical prices for GB00BDVZYZ77...
--------------------------------------------------
Extracting historical prices for GB0033986497...
--------------------------------------------------
Extracting historical prices for GG00BJL5FH87...
--------------------------------------------------
Extracting historical prices for GB0005800072...
--------------------------------------------------
Extracting historical prices for GB00BDFBVT43...
--------------------------------------------------
Extracting historical prices for GB00BY2Z0H74...
------

/var/folders/mv/kbtg0d9s34s4w1xjw4w31b0h0000gn/T/ipykernel_67464/3125937369.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.


In [191]:
ftse_rebal_data = historical_prices.reset_index().copy()

rm_isin = ["GB00BMMV6B79", "GB00BDD85M81", "GB0006886666", "GB0007990962", "GB00B8HX8Z88", "GB00BMMV6B79", "GB0005758098"]
lst_index = []
for i in range(len(rm_isin)):
    index = ftse_full_rebal.index[(ftse_full_rebal["ISIN\n"] == rm_isin[i])][0]
    lst_index.append(index)
    
rm_ftse_full_rebal = ftse_full_rebal.drop(lst_index)
rm_ftse_full_rebal = rm_ftse_full_rebal.reset_index()

lst_isin = list(rm_ftse_full_rebal["ISIN\n"])
lst_date = list(rm_ftse_full_rebal["post_date"])

In [194]:
def get_price(df, isin, date):
    pre_five_pd = []
    pre_three_pd = []
    pre_one_pd = []
    post_three_pd = []
    post_five_pd = []
    post_ten_pd = []
    post_twenty_pd = []
    merge_lst = []
    
    for i in range(len(isin)):
        #print(i)
        index = df.index[(df["trade_date"] == date[i]) & (df["isin_code"] == isin[i])][0]
        
        pre_five_pd.append(df["close"].iloc[index - 5])
        pre_three_pd.append(df["close"].iloc[index - 3])
        pre_one_pd.append(df["close"].iloc[index - 1])
        post_three_pd.append(df["close"].iloc[index + 3])
        post_five_pd.append(df["close"].iloc[index + 5])
        post_ten_pd.append(df["close"].iloc[index + 10])
        post_twenty_pd.append(df["close"].iloc[index + 20])
    merge_lst.append(pre_five_pd)
    merge_lst.append(pre_three_pd)
    merge_lst.append(pre_one_pd)
    merge_lst.append(post_three_pd)
    merge_lst.append(post_five_pd)
    merge_lst.append(post_ten_pd)
    merge_lst.append(post_twenty_pd)
    
    return merge_lst

In [200]:
ftse_tidy_data = pd.DataFrame()

ftse_tidy_data["Name"] = rm_ftse_full_rebal["Name"]
ftse_tidy_data["SEDOL"] = rm_ftse_full_rebal["SEDOL\n"]
ftse_tidy_data["ISIN"] = rm_ftse_full_rebal["ISIN\n"]
ftse_tidy_data["FTSE100"] = rm_ftse_full_rebal["FTSE100"]
ftse_tidy_data["FTSE250"] = rm_ftse_full_rebal["FTSE250"]
ftse_tidy_data["post_date"] = rm_ftse_full_rebal["post_date"]

lst_ftse_price = get_price(ftse_rebal_data, lst_isin, lst_date)

In [201]:
ftse_tidy_data["pre_five_pd"] = lst_ftse_price[0]
ftse_tidy_data["pre_three_pd"] = lst_ftse_price[1]
ftse_tidy_data["pre_one_pd"] = lst_ftse_price[2]
ftse_tidy_data["post_three_pd"] = lst_ftse_price[3]
ftse_tidy_data["post_five_pd"] = lst_ftse_price[4]
ftse_tidy_data["post_ten_pd"] = lst_ftse_price[5]
ftse_tidy_data["post_twenty_pd"] = lst_ftse_price[6]


In [203]:
ftse_tidy_data["diff_five_three"] = ftse_tidy_data["post_three_pd"] - ftse_tidy_data["pre_five_pd"]
ftse_tidy_data["diff_five_five"] = ftse_tidy_data["post_five_pd"] - ftse_tidy_data["pre_five_pd"]
ftse_tidy_data["diff_five_ten"] = ftse_tidy_data["post_ten_pd"] - ftse_tidy_data["pre_five_pd"]
ftse_tidy_data["diff_five_twenty"] = ftse_tidy_data["post_twenty_pd"] - ftse_tidy_data["pre_five_pd"]

ftse_tidy_data["diff_three_three"] = ftse_tidy_data["post_three_pd"] - ftse_tidy_data["pre_three_pd"]
ftse_tidy_data["diff_three_five"] = ftse_tidy_data["post_five_pd"] - ftse_tidy_data["pre_three_pd"]
ftse_tidy_data["diff_three_ten"] = ftse_tidy_data["post_ten_pd"] - ftse_tidy_data["pre_three_pd"]
ftse_tidy_data["diff_three_twenty"] = ftse_tidy_data["post_twenty_pd"] - ftse_tidy_data["pre_three_pd"]


ftse_tidy_data["diff_one_three"] = ftse_tidy_data["post_three_pd"] - ftse_tidy_data["pre_one_pd"]
ftse_tidy_data["diff_one_five"] = ftse_tidy_data["post_five_pd"] - ftse_tidy_data["pre_one_pd"]
ftse_tidy_data["diff_one_ten"] = ftse_tidy_data["post_ten_pd"] - ftse_tidy_data["pre_one_pd"]
ftse_tidy_data["diff_one_twenty"] = ftse_tidy_data["post_twenty_pd"] - ftse_tidy_data["pre_one_pd"]



In [204]:
ftse_tidy_data.describe()

,FTSE100,FTSE250,post_date,pre_five_pd,pre_three_pd,pre_one_pd,post_three_pd,post_five_pd,post_ten_pd,post_twenty_pd,...,diff_five_ten,diff_five_twenty,diff_three_three,diff_three_five,diff_three_ten,diff_three_twenty,diff_one_three,diff_one_five,diff_one_ten,diff_one_twenty
count,20.000000,66.000000,66,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,...,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000
mean,0.100000,0.060606,2022-11-23 02:32:43.636363520,4.936210,5.115649,5.020407,4.922321,4.868374,5.062907,4.894292,...,0.126697,-0.041918,-0.193329,-0.247276,-0.052742,-0.221358,-0.098087,-0.152034,0.042500,-0.126116
min,-1.000000,-1.000000,2022-06-20 00:00:00,0.250200,0.251400,0.254000,0.282600,0.279200,0.275000,0.278200,...,-11.940000,-26.020000,-2.899995,-3.500000,-3.000000,-26.639996,-3.500000,-4.100005,-3.600005,-25.760008
25%,-1.000000,-1.000000,2022-09-20 00:00:00,1.054000,1.146250,1.094750,1.074750,1.053500,1.141500,1.056000,...,-0.261250,-0.642750,-0.262250,-0.250500,-0.210001,-0.329000,-0.169500,-0.171499,-0.121500,-0.255500
50%,1.000000,1.000000,2022-09-20 00:00:00,2.062500,2.102500,2.047500,2.022500,1.960000,2.074500,2.235000,...,-0.036250,-0.013900,-0.043750,-0.071500,-0.059750,-0.085000,-0.022000,-0.037900,-0.011500,-0.086500
75%,1.000000,1.000000,2023-03-20 00:00:00,6.172498,5.772498,5.635998,5.487999,5.464999,6.526249,5.427498,...,0.117497,0.965750,-0.008250,-0.002000,0.011875,0.094000,0.017750,0.024275,0.051875,0.110700
max,1.000000,1.000000,2023-06-19 00:00:00,38.699996,38.500000,39.100005,35.600005,35.000000,35.500000,35.000000,...,16.042005,27.364996,1.029998,0.648001,11.971000,27.329996,0.680000,0.620999,11.991000,27.329996
std,1.020836,1.005811,NaN,6.724902,6.931511,6.849729,6.509971,6.459906,6.471283,6.486611,...,4.069055,6.926537,0.602846,0.682176,1.662924,6.000309,0.492299,0.605325,1.608523,5.899358


In [205]:
ftse_tidy_data.to_csv(
            f'/Users/abigail/Desktop/SMU/QF603/output/ftse_price_data.csv',
            index = False
            )